## Generative PCA
Most machine learning algorithms can be viewed from (at least) two perspectives: linear algebra, and stochastics, although both are closely related. The implications are quite different though. In the former lessons, we introduced PCA as a projective technique that finds principal directions and projects the original samples onto a lower-dimensional subspace in a way that maximizes the variance captured by the projected samples. These projections form the **latent space** of the dataset, which is a usually lower-dimensional representation of the data, and from which the original data can be reconstructed approximately. Latent spaces are encountered in many other machine learning algorithms, such as autoencoders or generative adversarial networks. The idea is always the same: 

*The latent space of a problem encodes usually not interpretable, but meaningful information about a higher-dimensional problem.*

This property has powerful implications. It enables a probabilistic perspective of the algorithms which turn from projection and transformation algorithms into modelling the probability distribution underlying the actual dataset, from which many things can be learned about the process behind the data. For example, the latent space vectors might be Gauss-distributed, such that outliers in the original dataset produce latent vectors that are far outside of the Gaussian-distributed latent space, enabling *anomaly detection*. 

We could also perform Bayesian model comparison, by making use of the marginal likelihood (see lectures 02 and 03), but we'll omit this here. The (probably) more interesting part is that probability distributions can be sampled and hence, be used to *generate* new data points that are, in a certain sense, *similar* to the original samples. Let's take a step back first. We saw that we can construct the latent space representation of a sample $x_i$ with the reduction matrix $R$ like so:

\begin{equation*}
  x_i^\prime = z_ = R^\text{T}x_i
\end{equation*}

Usually, latent space vectors are denoted as $z$. We can also reconstruct the original sample by aplying $R$:

\begin{equation*}
  \tilde{x}_i = RR^\text{T}x_i
\end{equation*}

where $\tilde{x}_i$ is the approximate reconstruction of $x_i$, since by projecting onto the principal directions, we lost the information about the directions with lower variances.

<img src="img/gPCA.png" width="400">

(*side note: PCA can be expressed as an optimization problem by demanding that the reconstruction $\tilde{X}$ of the data matrix should be as close as possible to $X$:*

\begin{equation*}
  \text{PCA}(X) = \underset{\tilde{X}}{\operatorname{arg\,min}} || \tilde{X} - X ||_\text{F}
\end{equation*}

*under the constraint $\text{rank}(\tilde{X}) = K$, where $K$ is the number of singular values included in the reduction (which is equivalent to $\dim(Z)$. $\text{F}$ denotes the Frobenius norm. The Eckard-Young theorem proves that the truncated SVD is the optimal solution to this problem.*)

What we can do now is to take a latent representation $z$ of some sample and change its components slightly. This will yield new data points that are in a meaningful way similar to the original samples, but not contained in the original dataset.

The probabilistic way to see this is that PCA models the probability to observe the sample $x_i$, given its latent representation $z$, the reduction matrix $R$, mean vector $\mu$ and covariance matrix $\Sigma$:

\begin{equation*}
  p(x|z,R,\mu,\Sigma) = \mathcal{N}(x|Rz+\mu,\lambda\mathbb{1})
\end{equation*}

where $\mathcal{N}$ is the normal distribution and $\lambda \mathbb{1}$ the covariance matrix in the latent space, which is diagonal and consisting of the eigenvalues of the data covariance matrix that survived the truncation.

We will explore how to use this generative method in the exercises today for creating and manipulating airfoil designs that will look like this:

In [1]:
import pickle
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

with open('af_pca.pkl', 'rb') as f:
    af_pca = pickle.load(f)
    
with open('components.pkl', 'rb') as f:
    components = pickle.load(f)

latent_dim = 10
    
def plot_generated(**kwargs):
    fig = plt.figure(figsize=(7,7))
    
    input_vector = [weight for weight in kwargs.values()]
    
    generated = af_pca.inverse_transform(input_vector)
    
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.15, 0.2])
    
    fig.patch.set_visible(False)
    plt.axis('off')
    
    plt.plot(generated.reshape(2,101)[0], generated.reshape(2,101)[1], lw=6, color="black")
    
base_sample = 8
component_sliders = [widgets.FloatSlider(
        value=components[base_sample][i],
        min=min(components[:,i]),
        max=max(components[:,i]),
        step=(max(components[:,i] - min(components[:,i]))/10),
    ) for i in range(latent_dim)]

kwargs = {'c' + str(i) : slider for i,slider in enumerate(component_sliders)}

interact(plot_generated, **kwargs)

interactive(children=(FloatSlider(value=-0.03723766920148769, description='c0', max=0.19187495785128805, min=-…

<function __main__.plot_generated(**kwargs)>